In [24]:
import pandas as pd
import json
import os
import openai
import time
import logging
from dotenv import load_dotenv
from loguru import logger
import tqdm
import random
from typing import List, Dict, Any

# 環境変数をロード
load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]

In [6]:
dataset_path = "../data/processed/annotated/laptops-trial_annotated_20251117_131424_annotated_20251117_141634.json"

output_path = "results/pref_json/laptops-trial_preferences.json"

## 実験

### データ準備

In [4]:
def load_and_split_data(file_path):
    # 1. JSONファイルを読み込む
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # データの整合性チェック（念のためリストかどうか確認）
    if not isinstance(data, list):
        raise ValueError("JSONのルート要素はリストである必要があります。")
    
    # データ数が足りているか確認（オプション）
    if len(data) < 30:
        print(f"警告: データ数が30未満です (現在の数: {len(data)})")

    # 2. 再現性のためにシードを固定（必要なければ削除可）
    random.seed(42)

    # 3. データをシャッフルする
    # 元のリストの順番が変わるため、必要なら data.copy() を使ってください
    random.shuffle(data)

    # 4. データを分割する
    # few_shot用: 最初の10個
    few_shot_data = data[:10]
    
    # 正解データ用: 残りの全て (30個なら残り20個)
    ground_truth_data = data[10:]

    return few_shot_data, ground_truth_data

In [8]:
try:
    few_shot_data, ground_truth_data = load_and_split_data(dataset_path)
    # 結果の確認
    print(f"全データ数: {len(few_shot_data) + len(ground_truth_data)}")
    print(f"Few-shot用データ数: {len(few_shot_data)}")
    print(f"正解データ数: {len(ground_truth_data)}")
    
    # 中身の確認（最初の1つだけ表示など）
    print(few_shot_data[0])

except FileNotFoundError:
    print("ファイルが見つかりませんでした。パスを確認してください。")
except Exception as e:
    print(f"エラーが発生しました: {e}")

全データ数: 30
Few-shot用データ数: 10
正解データ数: 20
{'sentence_id': 225, 'original_text': "Dells are ok, HPs aren't that good, but Macs or Fantastic.", 'translated_text': 'デルはまあまあ、HPはあまり良くないけど、Macは最高だ。', 'annotations': []}


In [15]:
test_inputs = [
    {
        "sentence_id": item["sentence_id"],
        "original_text": item["original_text"],
    }
    for item in ground_truth_data
]

print(test_inputs[:2])  # 最初の2つを表示して確認

[{'sentence_id': 2443, 'original_text': 'Boots up fast and runs great!'}, {'sentence_id': 2817, 'original_text': 'Bought it to use mostly for oline classes.'}]


### プロンプト

In [16]:
system_prompt = """
# Role
あなたはユーザーの潜在的な意図や感情を深く理解する高度な言語解析AIです。以下の手順に従って、入力されたテキストデータから発話者の「嗜好（Preference）」を抽出してください。

# Process
各入力文に対して以下のプロセスを実行し、**単一のJSONリスト**として出力してください。テキストのみの解説は不要です。

1. **Analyze (分析)**: 
   - **スキーマ適合性**: 嗜好スキーマ定義に照らし合わせ、入力文に含まれる該当可能な嗜好をすべて網羅的に特定。
   - **文脈推定**: 発話状況（レビュー、SNS、会話など）の推定。
   - **潜在的嗜好**: 明示的な言葉の背後にある価値観や潜在的欲求の分析。
   -> *この分析結果を要約し、出力JSONの "report" フィールドに格納してください。*

2. **Structure (構造化)**: 
   - 分析に基づき、嗜好データを "annotations" リストに格納してください。

---

# Criteria: 嗜好スキーマと制約 (厳守)

## 1. Classification & Sub-classification
(classification -> sub_classification)
- **liking**: [aesthetic/sensory, stimulation, identification]
- **wanting**: [interest, goal]
- **need**: [functional, social, personal]

## 2. その他の属性
- **polarity**: [positive, neutral, negative]
- **intensity**: [low, mid, high]
- **explicitness**: [explicit, implicit]

---

# JSON Output Format
出力は以下の形式のリストのみを含むこと。Markdown記法（```json等）は含めないでください。

[
  {
    "sentence_id": 81,
    "original_text": "Lightweight and the screen is beautiful!",
    "report": "【スキーマ適合性】「美しい」はLiking(aesthetic/sensory)、「軽量」はNeed(functional)に適合する。... (以下分析文)",
    "annotations": [
      {
        "entity": "laptop",
        "aspect": "screen",
        "polarity": "positive",
        "intensity": "mid",
        "classification": "liking",
        "sub_classification": "aesthetic/sensory",
        "context": "PCのレビュー",
        "explicitness": "explicit",
        "timestamp": "2025-11-17T14:16:34.582275"
      }
    ]
  }
]
"""

In [18]:
def analyze_preferences(
    test_inputs: List[Dict[str, Any]], 
    few_shot_data: List[Dict[str, Any]], 
    system_prompt: str
) -> List[Dict[str, Any]]:
    """
    gpt-4o-miniを使用して嗜好分析を行い、report付きのJSONを取得する
    """
    
    # Few-shotデータを文字列化してプロンプトに埋め込む準備
    # Few-shotにはreportキーが含まれていない可能性があるため、
    # モデルには「これは過去の正解データ（annotationsの例）である」と伝えます。
    few_shot_str = json.dumps(few_shot_data, ensure_ascii=False, indent=2)
    target_input_str = json.dumps(test_inputs, ensure_ascii=False, indent=2)

    # ユーザーメッセージの構築
    user_message = f"""
Here are some examples of correct annotations (Few-shot data):
{few_shot_str}

Now, please analyze the following inputs. 
Remember to include the "report" field in your output for the new inputs as instructed in the system prompt.

Target Inputs:
{target_input_str}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.0, # 分析の一貫性を保つため0推奨
            response_format={"type": "json_object"} # JSON出力を強制
        )

        # レスポンスの取得
        content = response.choices[0].message.content
        
        # JSONパースを試みる
        # gpt-4o-miniのjson_objectモードは {"key": ...} の形を返すことが多いため、
        # ルートがリストかオブジェクトかを確認して整形する
        try:
            parsed_data = json.loads(content)
            
            # response_format={"type": "json_object"}を使った場合、
            # モデルはしばしば {"results": [...]} や {"data": [...]} のようなキーでラップして返します。
            # リストが直接返ってこない場合のハンドリング
            if isinstance(parsed_data, dict):
                # 辞書の値の中からリストを探す
                for key, value in parsed_data.items():
                    if isinstance(value, list):
                        return value
                # リストが見つからない場合はそのまま返す（要確認）
                return [parsed_data]
            elif isinstance(parsed_data, list):
                return parsed_data
            
        except json.JSONDecodeError:
            print("JSONデコードエラー: モデルの出力を解析できませんでした。")
            print("Raw Content:", content)
            return []

    except Exception as e:
        print(f"API呼び出しエラー: {e}")
        return []

### 生成

In [25]:
client = openai.OpenAI()

results = analyze_preferences(
        test_inputs=test_inputs,
        few_shot_data=few_shot_data,
        system_prompt=system_prompt
)

# 結果の表示
print(json.dumps(results, indent=2, ensure_ascii=False))

# 必要であればファイルに保存
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

[
  {
    "sentence_id": 2443,
    "original_text": "Boots up fast and runs great!",
    "report": "【スキーマ適合性】「速く起動する」と「素晴らしく動作する」はLiking(stimulation)とNeed(functional)に適合する。発話者はパフォーマンスに満足していることを示唆している。",
    "annotations": [
      {
        "entity": "laptop",
        "aspect": "boot speed",
        "polarity": "positive",
        "intensity": "high",
        "classification": "liking",
        "sub_classification": "stimulation",
        "context": "laptopのパフォーマンスに関するレビュー",
        "explicitness": "explicit",
        "timestamp": "2025-11-17T14:16:34.582275"
      },
      {
        "entity": "laptop",
        "aspect": "performance",
        "polarity": "positive",
        "intensity": "high",
        "classification": "liking",
        "sub_classification": "stimulation",
        "context": "laptopのパフォーマンスに関するレビュー",
        "explicitness": "explicit",
        "timestamp": "2025-11-17T14:16:34.582275"
      }
    ]
  },
  {
    "sentence_id": 2817,
    "original_text": "Bought it to u

FileNotFoundError: [Errno 2] No such file or directory: 'results/pref_json/laptops-trial_preferences.json'

In [26]:
# 1. ディレクトリ部分だけを取り出す
output_dir = os.path.dirname(output_path)

# 2. ディレクトリパスがあり、かつ存在しない場合は作成する
# exist_ok=True にしておくと、既に存在していてもエラーになりません
if output_dir:
    os.makedirs(output_dir, exist_ok=True)

# 3. ファイル書き込み (元の処理)
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

In [27]:
results

[{'sentence_id': 2443,
  'original_text': 'Boots up fast and runs great!',
  'report': '【スキーマ適合性】「速く起動する」と「素晴らしく動作する」はLiking(stimulation)とNeed(functional)に適合する。発話者はパフォーマンスに満足していることを示唆している。',
  'annotations': [{'entity': 'laptop',
    'aspect': 'boot speed',
    'polarity': 'positive',
    'intensity': 'high',
    'classification': 'liking',
    'sub_classification': 'stimulation',
    'context': 'laptopのパフォーマンスに関するレビュー',
    'explicitness': 'explicit',
    'timestamp': '2025-11-17T14:16:34.582275'},
   {'entity': 'laptop',
    'aspect': 'performance',
    'polarity': 'positive',
    'intensity': 'high',
    'classification': 'liking',
    'sub_classification': 'stimulation',
    'context': 'laptopのパフォーマンスに関するレビュー',
    'explicitness': 'explicit',
    'timestamp': '2025-11-17T14:16:34.582275'}]},
 {'sentence_id': 2817,
  'original_text': 'Bought it to use mostly for online classes.',
  'report': '【スキーマ適合性】「オンラインクラス用に購入」はNeed(functional)に適合する。発話者は特定の目的のためにデバイスを使用する意図を示している。',
  'annotati

## 評価

In [32]:
import pandas as pd

def evaluate_preference_extraction(results, ground_truth_data):
    """
    分析結果(results)と正解データ(ground_truth_data)を比較し、
    完全一致スコアおよび属性ごとの正解率を算出する。
    
    Conditions:
    - Entity/Aspectのテキストは無視する。
    - One-to-Oneマッチング（一度マッチした予測データは消費される）。
    """
    
    # --- 1. データの整形 (sentence_id をキーにする) ---
    results_map = {item['sentence_id']: item.get('annotations', []) for item in results}
    
    # --- 2. カウンターの初期化 ---
    stats = {
        # 完全一致 (All 5 attributes match)
        "strict": {"tp": 0, "fp": 0, "fn": 0},
        
        # カテゴリ一致 (Classification & Sub-classification match)
        # これをベースに他の属性の精度を測る
        "category_match_count": 0,
        
        # 属性ごとの正解数 (カテゴリが一致しているペアの中でカウント)
        "attr_correct": {
            "polarity": 0,
            "intensity": 0,
            "explicitness": 0
        }
    }

    # --- 3. 評価ループ ---
    for gt_item in ground_truth_data:
        s_id = gt_item['sentence_id']
        gt_annots = gt_item.get('annotations', [])
        
        # この文に対する予測データのリスト（コピーして使用）
        pred_annots = results_map.get(s_id, [])[:] 
        
        # --- A. 完全一致 (Strict Match) の評価 ---
        # コピーをもう一つ作ってStrict判定用にする
        pred_annots_strict = pred_annots[:]
        
        for gt_obj in gt_annots:
            match_found = False
            
            for i, pred_obj in enumerate(pred_annots_strict):
                # 全5属性の比較
                keys = ['classification', 'sub_classification', 'polarity', 'intensity', 'explicitness']
                if all(gt_obj.get(k) == pred_obj.get(k) for k in keys):
                    match_found = True
                    pred_annots_strict.pop(i) # 消費
                    break
            
            if match_found:
                stats["strict"]["tp"] += 1
            else:
                stats["strict"]["fn"] += 1
        
        # 残ったものはFalse Positive
        stats["strict"]["fp"] += len(pred_annots_strict)

        # --- B. 属性ごとの精度 (Attribute-level Accuracy) の評価 ---
        # ここでは「嗜好の種類(Class/SubClass)」が合っているものをペアとみなして詳細評価する
        pred_annots_attr = pred_annots[:] # リセット
        
        for gt_obj in gt_annots:
            best_match_idx = -1
            
            # ClassificationとSub-Classificationが一致するものを探す
            for i, pred_obj in enumerate(pred_annots_attr):
                if (gt_obj.get('classification') == pred_obj.get('classification') and 
                    gt_obj.get('sub_classification') == pred_obj.get('sub_classification')):
                    best_match_idx = i
                    break
            
            if best_match_idx != -1:
                # カテゴリが一致したペアが見つかった
                stats["category_match_count"] += 1
                matched_pred = pred_annots_attr.pop(best_match_idx)
                
                # 各属性が一致しているかチェック
                for attr in ["polarity", "intensity", "explicitness"]:
                    if gt_obj.get(attr) == matched_pred.get(attr):
                        stats["attr_correct"][attr] += 1

    # --- 4. 指標の計算 ---
    
    # Strict Match Metrics
    tp = stats["strict"]["tp"]
    fp = stats["strict"]["fp"]
    fn = stats["strict"]["fn"]
    
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # Attribute-level Accuracy
    # 分母は「カテゴリ(Class+SubClass)が正解した数」とする
    denom = stats["category_match_count"]
    attr_accuracy = {}
    for attr, correct_count in stats["attr_correct"].items():
        attr_accuracy[attr] = correct_count / denom if denom > 0 else 0

    # --- 5. 結果の出力 ---
    print("=== 評価レポート ===")
    print(f"【完全一致 (Strict Match)】")
    print(f"  Precision : {precision:.3f} (予測のうち正解だった割合)")
    print(f"  Recall    : {recall:.3f} (正解のうち抽出できた割合)")
    print(f"  F1-Score  : {f1:.3f}")
    print(f"  (TP: {tp}, FP: {fp}, FN: {fn})")
    print("-" * 30)
    print(f"【属性ごとの正解率 (カテゴリ一致 {denom} 件中)】")
    print("  ※ 嗜好の種類(Class/SubClass)が正解だったペアのみで算出")
    for attr, score in attr_accuracy.items():
        print(f"  {attr.capitalize():<12}: {score:.3f}")
    
    return {
        "strict_metrics": {"precision": precision, "recall": recall, "f1": f1},
        "attribute_accuracy": attr_accuracy
    }

# --- 実行例 ---
# evaluate_preference_extraction(results, ground_truth_data)

In [33]:
evaluation_results = evaluate_preference_extraction(results, ground_truth_data)

evaluation_results

=== 評価レポート ===
【完全一致 (Strict Match)】
  Precision : 0.190 (予測のうち正解だった割合)
  Recall    : 0.308 (正解のうち抽出できた割合)
  F1-Score  : 0.235
  (TP: 4, FP: 17, FN: 9)
------------------------------
【属性ごとの正解率 (カテゴリ一致 7 件中)】
  ※ 嗜好の種類(Class/SubClass)が正解だったペアのみで算出
  Polarity    : 1.000
  Intensity   : 0.714
  Explicitness: 0.857


{'strict_metrics': {'precision': 0.19047619047619047,
  'recall': 0.3076923076923077,
  'f1': 0.23529411764705882},
 'attribute_accuracy': {'polarity': 1.0,
  'intensity': 0.7142857142857143,
  'explicitness': 0.8571428571428571}}

### 比較

In [34]:
def compare_results_visually(results, ground_truth_data):
    """
    分析結果(results)と正解データ(ground_truth_data)を
    IDごとに並べて表示し、目視比較するための関数
    """
    
    # 1. 高速検索のために正解データを sentence_id をキーにした辞書に変換
    gt_map = {item['sentence_id']: item for item in ground_truth_data}

    print(f"=== 比較レポート (全 {len(results)} 件) ===\n")

    for res_item in results:
        s_id = res_item['sentence_id']
        gt_item = gt_map.get(s_id)

        # --- ヘッダー情報 (原文・翻訳) ---
        print(f"🆔 ID: {s_id}")
        print(f"📄 原文: {res_item.get('original_text', 'N/A')}")
        # 翻訳文はresultsになければground_truth側から取得してみる
        trans_text = res_item.get('translated_text') or (gt_item.get('translated_text') if gt_item else 'N/A')
        print(f"🎌 翻訳: {trans_text}")
        print("-" * 50)

        # --- アノテーションの比較 ---
        
        # 分析結果 (Prediction)
        print("🤖 【 AI分析結果 (Prediction) 】")
        pred_annots = res_item.get('annotations', [])
        if pred_annots:
            print(json.dumps(pred_annots, indent=2, ensure_ascii=False))
        else:
            print("   [] (抽出なし)")

        print("") # 空行

        # 正解データ (Ground Truth)
        print("✅ 【 正解データ (Ground Truth) 】")
        if gt_item:
            gt_annots = gt_item.get('annotations', [])
            if gt_annots:
                print(json.dumps(gt_annots, indent=2, ensure_ascii=False))
            else:
                print("   [] (正解なし)")
        else:
            print("   (対応する正解データが見つかりません)")

        # 次のデータへの区切り線
        print("\n" + "=" * 80 + "\n")


In [35]:
# --- 実行 ---
compare_results_visually(results, ground_truth_data)

=== 比較レポート (全 20 件) ===

🆔 ID: 2443
📄 原文: Boots up fast and runs great!
🎌 翻訳: 起動が速くて、動作も快調！
--------------------------------------------------
🤖 【 AI分析結果 (Prediction) 】
[
  {
    "entity": "laptop",
    "aspect": "boot speed",
    "polarity": "positive",
    "intensity": "high",
    "classification": "liking",
    "sub_classification": "stimulation",
    "context": "laptopのパフォーマンスに関するレビュー",
    "explicitness": "explicit",
    "timestamp": "2025-11-17T14:16:34.582275"
  },
  {
    "entity": "laptop",
    "aspect": "performance",
    "polarity": "positive",
    "intensity": "high",
    "classification": "liking",
    "sub_classification": "stimulation",
    "context": "laptopのパフォーマンスに関するレビュー",
    "explicitness": "explicit",
    "timestamp": "2025-11-17T14:16:34.582275"
  }
]

✅ 【 正解データ (Ground Truth) 】
[
  {
    "entity": "laptop",
    "aspect": "Boots up",
    "polarity": "positive",
    "intensity": "mid",
    "classification": "need",
    "sub_classification": "functional",
    "cont